# Ejecución de un script de entrenamiento como un trabajo de comando

Puede usar el SDK de Python para Azure Machine Learning para enviar scripts como trabajos de comando. Mediante el uso de trabajos, puede realizar fácilmente un seguimiento de los parámetros y salidas de entrada al entrenar un modelo de Machine Learning.

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Seguimiento personalizado con MLflow

Al ejecutar un script como trabajo, puede usar MLflow en el script de entrenamiento para realizar un seguimiento del modelo. MLflow permite realizar un seguimiento de los parámetros, métricas o artefactos personalizados que quiera almacenar con la salida del trabajo.

Ejecute las celdas siguientes para crear el script **train-model-mlflow.py** en la carpeta **src**. El script entrena un modelo de clasificación mediante el archivo **diabetes.csv** de la misma carpeta, que se pasa como argumento. 

Revise el código siguiente para buscar que el script importará `mlflow` y registrará:

- Tasa de regularización como **parámetro**. 
- Precisión y AUC como **métricas**.
- Curva ROC trazada como **un artefacto**.

In [ ]:
Ahora, puede enviar el script como un trabajo de comando.

Ejecute la celda siguiente para entrenar el modelo. 

In [ ]:
En Studio, vaya al trabajo **diabetes-train-mlflow** para explorar la información general del trabajo de comando que ejecutó:

- Busque los parámetros registrados en la pestaña **Información general**, en **Parámetros**.
- Busque las métricas registradas en la pestaña **Métricas**.
- Busque los artefactos registrados en la pestaña **Imágenes** (específicamente para imágenes) y, en la pestaña **Salidas y registros** (todos los archivos).

## Registro automático con MLflow

En lugar de usar el registro personalizado, MLflow también puede registrar automáticamente los parámetros, las métricas y los artefactos. El registro automático con MLflow solo requiere una línea de código.

Ejecute la celda siguiente para crear el script **train-model-autolog.py** en la carpeta **src**. El script entrena un modelo de clasificación mediante el archivo **diabetes.csv** de la misma carpeta, que se pasa como argumento. 

Revise el código siguiente para buscar que el script importará `mlflow` y habilitará el registro automático con la línea: 

`mlflow.autolog()`

In [ ]:
Ahora, puede enviar el script como un trabajo de comando.

Ejecute la celda siguiente para entrenar el modelo. 

In [ ]:
En Studio, vaya al trabajo **diabetes-train-autolog** para explorar la información general del trabajo de comando que ejecutó:

- Busque los parámetros registrados en la pestaña **Información general**, en **Parámetros**.
- Busque las métricas registradas en la pestaña **Métricas**.
- Busque los artefactos registrados en la pestaña **Imágenes** (específicamente para imágenes) y, en la pestaña **Salidas y registros** (todos los archivos, incluidos los archivos del modelo).

## Uso de MLflow para ver y buscar experimentos

Azure Machine Learning Studio es una interfaz de usuario fácil de usar para ver y comparar ejecuciones de trabajos. Como alternativa, puede usar MLflow para ver los trabajos de experimento. 

Para enumerar los trabajos del área de trabajo, use el siguiente comando para enumerar los experimentos del área de trabajo:


In [ ]:
Para recuperar un experimento específico, puede obtenerlo por su nombre:

Con un nombre de experimento, puede recuperar todos los trabajos de ese experimento:

Para comparar más fácilmente las ejecuciones y salidas del trabajo, puede configurar la búsqueda para ordenar los resultados. Por ejemplo, la celda siguiente ordena los resultados por `start_time`, y solo muestra un máximo de `2` resultados: 

In [ ]:
Incluso puede crear una consulta para filtrar las ejecuciones. Las cadenas de consulta de filtro se escriben con una versión simplificada de la cláusula SQL `WHERE`. 

Para filtrar, puede usar dos clases de comparadores:

- Comparadores numéricos (métricas): =, !=, >, >=, < y <=.
- Comparadores de cadenas (parámetros, etiquetas y atributos): = y !=.

Más información sobre [cómo realizar un seguimiento de experimentos con MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

Now, you can submit the script as a command job.

Run the cell below to train the model. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In the Studio, navigate to the **diabetes-train-autolog** job to explore the overview of the command job you ran:

- Find the logged parameters in the **Overview** tab, under **Params**.
- Find the logged metrics in the **Metrics** tab.
- Find the logged artifacts in the **Images** tab (specifically for images), and in the **Outputs + logs** tab (all files, including the model files).

## Use MLflow to view and search for experiments

The Azure Machine Learning Studio is an easy-to-use UI to view and compare job runs. Alternatively, you can use MLflow to view experiment jobs. 

To list the jobs in the workspace, use the following command to list the experiments in the workspace:


In [ ]:
import mlflow
experiments = mlflow.list_experiments()
for exp in experiments:
    print(exp.name)

To retrieve a specific experiment, you can get it by its name:

In [ ]:
experiment_name = "diabetes-training"
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

Using an experiment name, you can retrieve all jobs of that experiment:

In [ ]:
mlflow.search_runs(exp.experiment_id)

To more easily compare job runs and outputs, you can configure the search to order the results. For example, the following cell orders the results by `start_time`, and only shows a maximum of `2` results: 

In [ ]:
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)

You can even create a query to filter the runs. Filter query strings are written with a simplified version of the SQL `WHERE` clause. 

To filter, you can use two classes of comparators:

- Numeric comparators (metrics): =, !=, >, >=, <, and <=.
- String comparators (params, tags, and attributes): = and !=.

Learn more about [how to track experiments with MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

In [ ]:
query = "metrics.AUC > 0.8 and tags.model_type = 'LogisticRegression'"
mlflow.search_runs(exp.experiment_id, filter_string=query)